In [1]:
import math 
import numpy as np 
import pandas as pd 
import random
from scipy.optimize import linprog

In [2]:
# class_prices = np.array([10, 20, 10, 5, 10, 30, 2, 23, 14, 31], dtype=np.int16).reshape(1, 10)
class_prices = np.ones((1, 10), dtype=np.int16)
classes = np.array(["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"])
bundles = {
    'B1':["a", "b", "c"],
    'B2':["f", "h", "h"],
    'B3':["j", "j", "j"],
    'B4':["d", "e", "i"],
    'B5':["f", "f", "g"],
    'B6':["a", "e", "i"],
    'B7':["b", "c", "d"],
    'B8':["g", "i", "j"]
}
bundles_discounts = {
    'B1': 0.1,
    'B2': 0.2,
    'B3':0.25,
    'B4':0.05,
    'B5':0.15,
    'B6': 0.35,
    'B7': 0.1,
    'B8':0.33
}

SyntaxError: invalid syntax (<ipython-input-2-a04f08a09106>, line 6)

In [5]:
bundle_discounts_arr = (1 - np.fromiter(bundles_discounts.values(), dtype=float)).reshape(8, 1)
bundle_discounts_arr

array([[0.9 ],
       [0.8 ],
       [0.75],
       [0.95],
       [0.85],
       [0.65],
       [0.9 ],
       [0.67]])

In [6]:
# not used for now 
# sorted bundle discounts (descending)
bundles_sorted_indices = np.argsort(np.fromiter(bundles_discounts.values(), dtype=float))
bundles_sorted_indices

array([3, 0, 6, 4, 1, 2, 7, 5])

In [7]:
basket_product_number = np.random.random_integers(1, 20, size=(10, ))
basket_product_number

array([ 8, 19,  3,  7, 17, 12,  5, 10,  3, 13])

In [8]:
from collections import Counter

def get_bundle_vector(bundle):
    vector = []
    for class_ in classes:
        if class_ in bundle:
            vector.append(Counter(bundle)[class_])
        else:
            vector.append(0)
    return vector

In [9]:
bundle_dict = {k: get_bundle_vector(v) for k, v in bundles.items()}
bundle_dict

{'B1': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 'B2': [0, 0, 0, 0, 0, 1, 0, 2, 0, 0],
 'B3': [0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 'B4': [0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
 'B5': [0, 0, 0, 0, 0, 2, 1, 0, 0, 0],
 'B6': [1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
 'B7': [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 'B8': [0, 0, 0, 0, 0, 0, 1, 0, 1, 1]}

In [10]:
bundle_matrix = pd.DataFrame.from_dict(bundle_dict, orient='index').values
bundle_matrix

array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 2, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1]])

In [11]:
# not used for now 
# if discount if bigger bungle goes first (descending)
bundle_sorted_matrix = bundle_matrix[bundles_sorted_indices][:]
bundle_sorted_matrix

array([[0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
       [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0]])

In [12]:
bundle_discounts_arr = (1 - np.fromiter(bundles_discounts.values(), dtype=float)).reshape(8, 1)
bundle_discounts_arr

array([[0.9 ],
       [0.8 ],
       [0.75],
       [0.95],
       [0.85],
       [0.65],
       [0.9 ],
       [0.67]])

In [178]:
# Not used for now
# prices for ALL classes with discounts  
# one row = one discount value 
# one column = one class
discounted_prices = bundle_discounts_arr * class_prices
discounted_prices

array([[0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 ],
       [0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 ],
       [0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75],
       [0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95],
       [0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85],
       [0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65],
       [0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 , 0.9 ],
       [0.67, 0.67, 0.67, 0.67, 0.67, 0.67, 0.67, 0.67, 0.67, 0.67]])

In [14]:
bundle_number = np.zeros((8, 1))
# [1,2,3,4,5,6,7,8]
# a = np.array(a).reshape((1,8))bundle_number
bundle_number

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [391]:
# TODO: add arguments 
def get_bundle_cost(bundel_num):
    current_discounts = (bundel_num > 0) * bundle_discounts_arr
    bundle_cost = np.asscalar(sum((bundel_num * bundle_matrix).dot(class_prices.T) * current_discounts))
    return bundle_cost


In [392]:
# TODO: add arguments 
def get_no_bundled_cost(bundel_num):
    no_bundled_product_number = basket_product_number - bundel_num.T.dot(bundle_matrix)

    not_enough_product_in_basket = ((basket_product_number - bundel_num.T.dot(bundle_matrix))< 0).any()
    if not_enough_product_in_basket:
        print('ALARM! NOT ENOUGHT PRODUCTS IN BASKET')

    return np.asscalar(
        (basket_product_number - bundel_num.T.dot(bundle_matrix)).dot(class_prices.T)
    )

In [393]:
# TODO: add arguments 
def get_total_cost(bundel_num):
    return round(get_no_bundled_cost(bundel_num) + get_bundle_cost(bundel_num), 2)

In [394]:
# bundle number = 0, no bundles 
bundle_number = np.zeros((8, 1))

total_cost_without_discount = get_total_cost(bundle_number)

manual_cost_without_discount = np.asscalar(class_prices.dot(basket_product_number.T))

assert total_cost_without_discount == manual_cost_without_discount, \
f'cost without discount is wrong, expected {manual_cost_without_discount}, got {total_cost_without_discount} '

print(f'Success! Total cost without bundles is {total_cost_without_discount}')

Success! Total cost without bundles is 97.0


In [395]:
# Cost without 1 bundle calculation 
# bundle number = 1, b1 is working 
bundle_number = np.vstack(([[1]], np.zeros((7, 1))))

total_cost_with_1_discount = get_no_bundled_cost(bundle_number)

manual_cost_with_1_discount = np.asscalar(class_prices.dot((basket_product_number - bundle_matrix[0]).T))

assert total_cost_with_1_discount == manual_cost_with_1_discount, \
f'cost with 1 bundle is wrong, expected {manual_cost_with_1_discount}, got {total_cost_with_1_discount} '

print(f'Success! Total cost with 1 bundle is {total_cost_with_1_discount}')

Success! Total cost with 1 bundle is 94.0


In [396]:
# Only discount (bundle) cost calculating
# bundle number = 1, b1 is working 
bundle_number = np.vstack(([[1]], np.zeros((7, 1))))

one_bundle_cost = get_bundle_cost(bundle_number)
manual_1_bundle_cost = np.asscalar(bundle_matrix[0].T.dot(class_prices.T) * bundle_discounts_arr[0])

assert one_bundle_cost == manual_1_bundle_cost, \
f'1 bundle cost is wrong, expected {one_bundle_cost}, got {one_bundle_cost} '

print(f'Success! 1 bundle cost is {one_bundle_cost}')

Success! 1 bundle cost is 2.7


In [397]:
# Only discount (bundle) cost calculating
# bundle number = 2, b1 and b2 are working 
bundle_number = np.vstack(([[1], [1]], np.zeros((6, 1))))

two_bundle_cost = get_bundle_cost(bundle_number)

manual_2_bundle_cost = np.asscalar(sum(bundle_matrix[:2].dot(class_prices.T) * bundle_discounts_arr[:2]))

assert two_bundle_cost == manual_2_bundle_cost, \
f'2 bundle cost is wrong, expected {two_bundle_cost}, got {manual_2_bundle_cost} '

print(f'Success! 1 bundle cost is {round(two_bundle_cost, 1)}')

Success! 1 bundle cost is 5.1


In [398]:
# Total cost calculation 
# bundle number = 2, b1 and b2 are working
bundle_number = np.vstack(([[1], [1]], np.zeros((6, 1))))

cost = get_total_cost(bundle_number)

number_left = basket_product_number - bundle_number.T.dot(bundle_matrix)
manual_without_2_bundle_cost = np.asscalar(class_prices.dot(number_left.T))
manual_2_bundle_cost = np.asscalar(sum(bundle_matrix[:2].dot(class_prices.T) * bundle_discounts_arr[:2]))
manual_cost = manual_without_2_bundle_cost + manual_2_bundle_cost

assert cost == manual_cost, \
f'Total cost with 2 bundles is wrong, expected {manual_cost}, got {cost} '

print(f'Success! Total cost with 2 bundles is {round(cost, 1)}')

Success! Total cost with 2 bundles is 96.1


In [288]:
# MAX -> z = k1 * x1 + k2 * x2 + ... + kn * xn 
# xn - number of bundles
# kn = 1 (?????)
obj = np.ones(bundle_number.shape)

# print(basket_product_number)
# print(bundle_number.T.dot(bundle_matrix))
# print(bundle_number.T.dot(bundle_matrix) <= basket_product_number)


# left-side coefficients from inequality 
# sum(kn * xn * bundle_matrix[n]) <= basket_product_number
# N.T.dot(X) <= M.T 
# k = ones 
product_number_for_bundles = bundle_number.T.dot(bundle_matrix)
lhs_ineq = bundle_matrix.T

rhs_ineq = basket_product_number.T

# xn >= 0 
bnd = np.array([(0, np.inf) for _ in range(bundle_number.shape[0])])
print(bnd)
# min(f(x)) = -max(-f(x))
opt = linprog(
    c=-obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd, method='revised simplex', options={"disp": True}
    )
opt


[[ 0. inf]
 [ 0. inf]
 [ 0. inf]
 [ 0. inf]
 [ 0. inf]
 [ 0. inf]
 [ 0. inf]
 [ 0. inf]]
Phase Iteration Minimum Slack       Constraint Residual Objective          
1     0         3.0                 0.0                 0.0                 
Phase Iteration Minimum Slack       Constraint Residual Objective          
2     0         3.0                 0.0                 0.0                 
2     1         0.0                 0.0                 -3.0                
2     2         0.0                 0.0                 -8.0                
2     3         0.0                 0.0                 -12.33333333333     
2     4         0.0                 0.0                 -15.33333333333     
2     5         0.0                 0.0                 -18.83333333333     
Optimization terminated successfully.
         Current function value: -18.833333  
         Iterations: 5


     con: array([], dtype=float64)
     fun: -18.833333333333332
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([ 5. , 16. ,  0. ,  4. , 14. ,  0. ,  1.5,  0. ,  0. ,  0. ])
  status: 0
 success: True
       x: array([3.        , 5.        , 4.33333333, 3.        , 3.5       ,
       0.        , 0.        , 0.        ])

In [256]:
opt = linprog(
    c=-obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd, method='interior-point', options={"disp": True}
    )
opt

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 -8.0                
0.2663333735582     0.2663333735582     0.2663333735582     0.7490021987658  0.2663333735582     -10.12916199003     
0.06344590492189    0.06344590492187    0.06344590492186    0.7777472339424  0.06344590492187    -13.61213206473     
0.01028983893067    0.01028983893067    0.01028983893067    0.8489405954292  0.01028983893067    -17.88648477847     
0.0006864660552709  0.0006864660552706  0.0006864660552705  0.9388791378118  0.0006864660552705  -18.75183767436     
3.346321523001e-07  3.346321522801e-07  3.34632152464e-07   0.9997597664385  3.34632152263e-07   -18.83330645011     
1.673143793307e-11  1.673159401948e-11  1.673150507031e-11  0.9999500000286  1.673161742878e-11  -18.83333333199     
Optimization terminated successfully.
         Current fu

     con: array([], dtype=float64)
     fun: -18.833333331989177
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([4.93965487e+00, 1.60000000e+01, 2.13876916e-10, 4.06034513e+00,
       1.40000000e+01, 6.62170763e-10, 1.50000000e+00, 1.12392229e-09,
       9.48245926e-11, 1.05011999e-09])
  status: 0
 success: True
       x: array([1.54627737e+00, 5.00000000e+00, 4.33333333e+00, 1.48593224e+00,
       3.50000000e+00, 1.51406776e+00, 1.45372263e+00, 2.20054083e-10])

In [251]:
opt = linprog(
    c=-obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd, method='simplex', options={"disp": True}
    )
opt

Optimization terminated successfully.
         Current function value: -18.833333  
         Iterations: 11


     con: array([], dtype=float64)
     fun: -18.833333333333332
 message: 'Optimization terminated successfully.'
     nit: 11
   slack: array([ 8. , 16. ,  0. ,  1. , 14. ,  0. ,  1.5,  0. ,  0. ,  0. ])
  status: 0
 success: True
       x: array([0.        , 5.        , 4.33333333, 3.        , 3.5       ,
       0.        , 3.        , 0.        ])

In [295]:
bundle_dict

{'B1': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 'B2': [0, 0, 0, 0, 0, 1, 0, 2, 0, 0],
 'B3': [0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 'B4': [0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
 'B5': [0, 0, 0, 0, 0, 2, 1, 0, 0, 0],
 'B6': [1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
 'B7': [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 'B8': [0, 0, 0, 0, 0, 0, 1, 0, 1, 1]}

In [399]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

# bundle_number = np.ones((1, 8))

bundle_n = [0, 1, 2, 3, 4, 5, 6, 7]

# list of decision varioables
bundle_lp = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']

bundle_dict = {k: get_bundle_vector(v) for k, v in bundles.items()}

discounts = {
    'B1': 0.9, 
    'B2': 0.8,
    'B3': 0.75,
    'B4': 0.95, 
    'B5': 0.85, 
    'B6': 0.65,
    'B7': 0.9, 
    'B8': 0.67
}

bun = np.zeros((8, 1))

raw_cost = get_no_bundled_cost(bun)


# MAXIMIZE number of bundels 
# Create the model
model = LpProblem(name="Find max bundles problem", sense=LpMaximize)
# Initialize the decision variables
x_vars = LpVariable.dicts('x', bundle_lp, lowBound=0, cat='Integer')

# Z = sum(b_i)
model += lpSum([x_vars[i] for i in bundle_lp])
# Restriction 
# number of products in bundles must be less or equal with product number in basket 
for p in range(len(classes)):
    model += lpSum([bundle_dict[b][p] * x_vars[b] for b in bundle_lp]) <= basket_product_number[p]

model

Find_max_bundles_problem:
MAXIMIZE
1*x_B1 + 1*x_B2 + 1*x_B3 + 1*x_B4 + 1*x_B5 + 1*x_B6 + 1*x_B7 + 1*x_B8 + 0
SUBJECT TO
_C1: x_B1 + x_B6 <= 8

_C2: x_B1 + x_B7 <= 19

_C3: x_B1 + x_B7 <= 3

_C4: x_B4 + x_B7 <= 7

_C5: x_B4 + x_B6 <= 17

_C6: x_B2 + 2 x_B5 <= 12

_C7: x_B5 + x_B8 <= 5

_C8: 2 x_B2 <= 10

_C9: x_B4 + x_B6 + x_B8 <= 3

_C10: 3 x_B3 + x_B8 <= 13

VARIABLES
0 <= x_B1 Integer
0 <= x_B2 Integer
0 <= x_B3 Integer
0 <= x_B4 Integer
0 <= x_B5 Integer
0 <= x_B6 Integer
0 <= x_B7 Integer
0 <= x_B8 Integer

In [400]:
model.solve() 
print(f"Status: {LpStatus[model.status]}")


check_bundle = np.array([])

for x in model.variables():
    print(f"{x.name} = {x.varValue}")
    check_bundle = np.append(check_bundle, x.varValue)

Status: Optimal
x_B1 = 3.0
x_B2 = 5.0
x_B3 = 4.0
x_B4 = 3.0
x_B5 = 3.0
x_B6 = 0.0
x_B7 = 0.0
x_B8 = 0.0


In [300]:
basket_product_number

array([ 8, 19,  3,  7, 17, 12,  5, 10,  3, 13])

In [301]:
bundle_dict

{'B1': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 'B2': [0, 0, 0, 0, 0, 1, 0, 2, 0, 0],
 'B3': [0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 'B4': [0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
 'B5': [0, 0, 0, 0, 0, 2, 1, 0, 0, 0],
 'B6': [1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
 'B7': [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 'B8': [0, 0, 0, 0, 0, 0, 1, 0, 1, 1]}

In [382]:
print(bundle_discounts_arr, bundle_discounts_arr.shape)
print(class_prices, class_prices.shape)

[[0.9 ]
 [0.8 ]
 [0.75]
 [0.95]
 [0.85]
 [0.65]
 [0.9 ]
 [0.67]] (8, 1)
[[1 1 1 1 1 1 1 1 1 1]] (1, 10)


In [402]:
check_bundle = check_bundle.reshape(1, 8).T
check_cost = get_total_cost(check_bundle)

check_cost

88.3

In [403]:
bun = np.zeros((8, 1))

print(get_no_bundled_cost(bun))

97.0
97
